In [2]:
import numpy as np
import pandas as pd

In [6]:
group_results = pd.read_csv('data/group_results.csv', encoding='utf-8')
group_results = group_results.drop('group', axis=1)
group_results.head()

,home_short_team,away_short_team,home_goals,away_goals
0,QAT,ECU,0,2
1,ENG,IRN,6,2
2,SEN,NED,0,2
3,USA,WAL,1,1
4,ARG,KSA,1,2


In [8]:
round_of_16_results = pd.read_csv('data/round_of_16_results.csv', encoding='utf-8')
round_of_16_results = round_of_16_results.drop('id_game', axis=1)
round_of_16_results.head()

,home_short_team,away_short_team,home_goals,away_goals
0,NED,USA,3,1
1,ARG,AUS,2,1
2,FRA,POL,3,1
3,ENG,SEN,3,0
4,JPN,CRO,1,1


In [14]:
results = pd.concat([group_results, round_of_16_results], ignore_index=True)
results.head()

,home_short_team,away_short_team,home_goals,away_goals
0,QAT,ECU,0,2
1,ENG,IRN,6,2
2,SEN,NED,0,2
3,USA,WAL,1,1
4,ARG,KSA,1,2


In [15]:
def winner(row):  
    if row['home_goals'] > row['away_goals']:
        return 1
    elif row['home_goals'] < row['away_goals']:
        return -1
    elif row['home_goals'] == row['away_goals']:
        return 0
    return -1000

In [16]:
results['winner'] = results.apply(lambda row: winner(row), axis=1)

In [17]:
results.head()

,home_short_team,away_short_team,home_goals,away_goals,winner
0,QAT,ECU,0,2,-1
1,ENG,IRN,6,2,1
2,SEN,NED,0,2,-1
3,USA,WAL,1,1,0
4,ARG,KSA,1,2,-1


In [18]:
teams = pd.read_csv('data/WC2022_teams.csv', encoding='utf-8')
teams.sort_values(by='rank', ascending=True, inplace=True)
teams.head()

,short_team,team,rank,points
24,BRA,Brazil,1,1841
20,BEL,Belgium,2,1817
8,ARG,Argentina,3,1774
12,FRA,France,4,1760
4,ENG,England,5,1728


In [19]:
df = pd.read_csv('data/international_matches.csv', encoding='utf-8')
df.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
most_recent_mask=(df.groupby('home_team')['date'].transform('max') == df['date'])

most_recent_df=df[most_recent_mask]

most_recent_df.shape

(211, 25)

In [21]:
wc_teams_mask=(most_recent_df['home_team'].isin(teams['team'].unique()))
wc_teams_df=most_recent_df[wc_teams_mask]

wc_teams_df.shape

(32, 25)

In [22]:
team_cols=['home_team', 'home_team_continent', 'home_team_goalkeeper_score', 'home_team_mean_defense_score', 'home_team_mean_offense_score',
            'home_team_mean_midfield_score']
wc_teams_df = wc_teams_df[team_cols]

wc_teams_df=wc_teams_df.rename(columns={'home_team':'team', 'home_team_continent':'team_continent', 'home_team_goalkeeper_score':'team_goalkeeper_score',
                    'home_team_mean_defense_score':'team_mean_defense_score', 'home_team_mean_offense_score':'team_mean_offense_score', 
                    'home_team_mean_midfield_score':'team_mean_midfield_score'})
wc_teams_df.head()

,team,team_continent,team_goalkeeper_score,team_mean_defense_score,team_mean_offense_score,team_mean_midfield_score
23497,Brazil,South America,89.0,84.8,86.3,85.5
23516,Cameroon,Africa,67.0,76.8,77.7,75.0
23615,Qatar,Asia,NaN,NaN,NaN,NaN
23656,Ghana,Africa,74.0,75.5,76.0,78.2
23667,Tunisia,Africa,NaN,70.8,72.3,74.0


In [23]:
wc2022_teams_df = pd.merge(teams,wc_teams_df,on='team')

wc2022_teams_df.rename(columns={'rank':'team_fifa_rank','points':'team_total_fifa_points'}, inplace=True)

wc2022_teams_df['team_rank_wc'] = range(1, len(wc2022_teams_df) + 1)

wc2022_teams_df

,short_team,team,team_fifa_rank,team_total_fifa_points,team_continent,team_goalkeeper_score,team_mean_defense_score,team_mean_offense_score,team_mean_midfield_score,team_rank_wc
0,BRA,Brazil,1,1841,South America,89.0,84.8,86.3,85.5,1
1,BEL,Belgium,2,1817,Europe,89.0,80.8,85.7,85.5,2
2,ARG,Argentina,3,1774,South America,84.0,82.2,89.0,84.0,3
3,FRA,France,4,1760,Europe,87.0,84.2,88.3,86.8,4
4,ENG,England,5,1728,Europe,83.0,85.0,88.0,84.0,5
5,ESP,Spain,7,1715,Europe,84.0,86.5,85.0,86.0,6
6,NED,Netherlands,8,1695,Europe,81.0,85.2,83.0,83.5,7
7,POR,Portugal,9,1677,Europe,82.0,85.2,86.0,84.5,8
8,DEN,Denmark,10,1667,Europe,85.0,80.2,77.7,80.2,9
9,GER,Germany,11,1650,Europe,90.0,84.0,82.7,87.8,10


In [24]:
wc2022_teams_df.groupby('team_continent')['team_fifa_rank'].mean().sort_values(ascending=True)

team_continent
Europe           11.461538
South America    15.500000
North America    25.250000
Asia             34.600000
Africa           34.800000
Oceania          38.000000
Name: team_fifa_rank, dtype: float64

In [25]:
def continent_wc_rank(row):
    if row['team_continent'] == 'Europe':
        return 1
    elif row['team_continent'] == 'South America':
        return 2
    elif row['team_continent'] == 'North America':
        return 3
    elif row['team_continent'] == 'Asia':
        return 4
    elif row['team_continent'] == 'Africa':
        return 5
    else:
        return 6

In [26]:
wc2022_teams_df['continent_wc_rank'] = wc2022_teams_df.apply(lambda row: continent_wc_rank(row), axis=1)
wc2022_teams_df.head()

,short_team,team,team_fifa_rank,team_total_fifa_points,team_continent,team_goalkeeper_score,team_mean_defense_score,team_mean_offense_score,team_mean_midfield_score,team_rank_wc,continent_wc_rank
0,BRA,Brazil,1,1841,South America,89.0,84.8,86.3,85.5,1,2
1,BEL,Belgium,2,1817,Europe,89.0,80.8,85.7,85.5,2,1
2,ARG,Argentina,3,1774,South America,84.0,82.2,89.0,84.0,3,2
3,FRA,France,4,1760,Europe,87.0,84.2,88.3,86.8,4,1
4,ENG,England,5,1728,Europe,83.0,85.0,88.0,84.0,5,1


In [27]:
def df_preparation(df):
    output_df = pd.merge(df, wc2022_teams_df.add_prefix('home_'), on='home_short_team')
    output_df = pd.merge(output_df, wc2022_teams_df.add_prefix('away_'), on='away_short_team')
    output_df.drop(['home_team', 'away_team'], axis=1, inplace=True)
    return output_df

In [28]:
results = df_preparation(results)

results.drop(['home_goals', 'away_goals'], axis=1, inplace=True)
results.head()

,home_short_team,away_short_team,winner,home_team_fifa_rank,home_team_total_fifa_points,home_team_continent,home_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,...,home_continent_wc_rank,away_team_fifa_rank,away_team_total_fifa_points,away_team_continent,away_team_goalkeeper_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,away_team_rank_wc,away_continent_wc_rank
0,QAT,ECU,-1,50,1440,Asia,NaN,NaN,NaN,NaN,...,4,44,1464,South America,71.0,73.5,76.0,74.5,29,2
1,NED,ECU,0,8,1695,Europe,81.0,85.2,83.0,83.5,...,1,44,1464,South America,71.0,73.5,76.0,74.5,29,2
2,QAT,SEN,-1,50,1440,Asia,NaN,NaN,NaN,NaN,...,4,18,1584,Africa,83.0,79.0,80.7,79.0,16,5
3,ENG,SEN,1,5,1728,Europe,83.0,85.0,88.0,84.0,...,1,18,1584,Africa,83.0,79.0,80.7,79.0,16,5
4,ECU,SEN,-1,44,1464,South America,71.0,73.5,76.0,74.5,...,2,18,1584,Africa,83.0,79.0,80.7,79.0,16,5


In [29]:
(results.isnull().sum()/results.shape[0]).sort_values(ascending=True) * 100

home_short_team                  0.000000
away_team_mean_midfield_score    0.000000
away_team_mean_offense_score     0.000000
away_team_mean_defense_score     0.000000
away_team_continent              0.000000
away_team_total_fifa_points      0.000000
away_team_fifa_rank              0.000000
home_continent_wc_rank           0.000000
away_team_rank_wc                0.000000
home_team_rank_wc                0.000000
home_team_continent              0.000000
home_team_total_fifa_points      0.000000
home_team_fifa_rank              0.000000
winner                           0.000000
away_short_team                  0.000000
away_continent_wc_rank           0.000000
away_team_goalkeeper_score       1.785714
home_team_mean_offense_score     5.357143
home_team_mean_defense_score     5.357143
home_team_mean_midfield_score    5.357143
home_team_goalkeeper_score       8.928571
dtype: float64

In [30]:
results.min()

home_short_team                     ARG
away_short_team                     ARG
winner                               -1
home_team_fifa_rank                   1
home_team_total_fifa_points        1393
home_team_continent              Africa
home_team_goalkeeper_score         67.0
home_team_mean_defense_score       69.0
home_team_mean_offense_score       67.7
home_team_mean_midfield_score      69.0
home_team_rank_wc                     1
home_continent_wc_rank                1
away_team_fifa_rank                   1
away_team_total_fifa_points        1393
away_team_continent              Africa
away_team_goalkeeper_score         67.0
away_team_mean_defense_score       69.0
away_team_mean_offense_score       67.7
away_team_mean_midfield_score      69.0
away_team_rank_wc                     1
away_continent_wc_rank                1
dtype: object

In [31]:
def fill_empty(df):

    fill0 = ['home_team_goalkeeper_score','away_team_goalkeeper_score','home_team_mean_defense_score','home_team_mean_offense_score',
            'home_team_mean_midfield_score','away_team_mean_defense_score','away_team_mean_offense_score','away_team_mean_midfield_score']

    for col in fill0:
        df[col].fillna(50, inplace=True)
        
    return df

In [32]:
results = fill_empty(results)
results.drop(['home_short_team', 'away_short_team', 'home_team_continent','away_team_continent'], axis=1, inplace=True)
results.head()

,winner,home_team_fifa_rank,home_team_total_fifa_points,home_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,home_team_rank_wc,home_continent_wc_rank,away_team_fifa_rank,away_team_total_fifa_points,away_team_goalkeeper_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,away_team_rank_wc,away_continent_wc_rank
0,-1,50,1440,50.0,50.0,50.0,50.0,30,4,44,1464,71.0,73.5,76.0,74.5,29,2
1,0,8,1695,81.0,85.2,83.0,83.5,7,1,44,1464,71.0,73.5,76.0,74.5,29,2
2,-1,50,1440,50.0,50.0,50.0,50.0,30,4,18,1584,83.0,79.0,80.7,79.0,16,5
3,1,5,1728,83.0,85.0,88.0,84.0,5,1,18,1584,83.0,79.0,80.7,79.0,16,5
4,-1,44,1464,71.0,73.5,76.0,74.5,29,2,18,1584,83.0,79.0,80.7,79.0,16,5


In [33]:
results.shape

(56, 17)

In [34]:
from sklearn.model_selection import train_test_split

X = results.drop('winner', axis=1)
y = results['winner']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [35]:
from sklearn.preprocessing import RobustScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif 
model = make_pipeline(RobustScaler(), SelectKBest(f_classif, k=16), LogisticRegression(max_iter=1000))

In [36]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve

In [37]:
def evaluation(model):

    model.fit(X_train, y_train)
        
    ypred = model.predict(X_test)
        
    print(confusion_matrix(y_test, ypred))
    print(classification_report(y_test, ypred))

In [38]:
evaluation(model)

[[4 0 1]
 [0 0 2]
 [0 0 5]]
              precision    recall  f1-score   support

          -1       1.00      0.80      0.89         5
           0       0.00      0.00      0.00         2
           1       0.62      1.00      0.77         5

    accuracy                           0.75        12
   macro avg       0.54      0.60      0.55        12
weighted avg       0.68      0.75      0.69        12



c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [39]:
from sklearn.model_selection import GridSearchCV

In [40]:
hyper_params = {'logisticregression__C': [1, 10, 100, 1000],
                'logisticregression__penalty' : ['l1', 'l2'],
                'logisticregression__solver' : ['liblinear'],
                'selectkbest__k': range(2,29)}

In [41]:
grid = GridSearchCV(model, hyper_params, scoring='f1_weighted', cv=4)
grid.fit(X_train, y_train)
print(grid.best_params_)
y_pred = grid.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))

{'logisticregression__C': 100, 'logisticregression__penalty': 'l1', 'logisticregression__solver': 'liblinear', 'selectkbest__k': 3}
              precision    recall  f1-score   support

          -1       0.83      1.00      0.91         5
           0       0.33      0.50      0.40         2
           1       1.00      0.60      0.75         5

    accuracy                           0.75        12
   macro avg       0.72      0.70      0.69        12
weighted avg       0.82      0.75      0.76        12



c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
384 fits failed out of a total of 864.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\sk

In [67]:
quarter_finals = pd.read_csv('data/quarter_finals.csv', encoding='utf-8')

quarter_finals.head()

,home_short_team,away_short_team,id_game
0,ARG,NED,57
1,BRA,CRO,58
2,FRA,ENG,59
3,POR,MAR,60


In [68]:
def calculate_probas(df):

    df = df_preparation(df)

    df = fill_empty(df)
    
    df.drop(['home_team_continent','away_team_continent'], axis=1, inplace=True)

    cols = grid.best_estimator_.named_steps['selectkbest'].get_support(indices=True)
    cols_names = df.drop(['home_short_team', 'away_short_team', 'id_game'], axis=1).iloc[:,cols].columns

    
    probs = grid.best_estimator_.predict_proba(df.drop(['home_short_team', 'away_short_team', 'id_game'], axis=1))

    d = {}

    for col in ['home_short_team', 'away_short_team', 'id_game']:
        d.update({col: df[col]})

    d2 = {'prob_win': probs[:,2], 'prob_draw': probs[:,1], 'prob_lose': probs[:,0]}

    d.update(d2)

    scores = pd.DataFrame(data=d)

    return scores


In [69]:
scores_quarter = calculate_probas(quarter_finals)

In [70]:
scores_quarter

,home_short_team,away_short_team,id_game,prob_win,prob_draw,prob_lose
0,ARG,NED,57,0.734794,0.118563,0.146643
1,BRA,CRO,58,0.676235,0.219804,0.103961
2,FRA,ENG,59,0.669027,0.222059,0.108914
3,POR,MAR,60,0.492231,0.413821,0.093948


In [62]:
scores_quarter

,home_short_team,away_short_team,id_game,prob_win,prob_draw,prob_lose
0,NED,ARG,57,0.427389,0.471534,0.101078
1,CRO,BRA,58,0.383954,0.244003,0.372043
2,ENG,FRA,59,0.605574,0.299989,0.094436
3,MAR,POR,60,0.373400,0.421395,0.205205
